In [1]:
from gssutils import *

scraper = Scraper('https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Pages/OTSReports.aspx')
scraper

## HM Revenue & Customs uktradeinfo - OTS Reports

This is a catalog of datasets; choose one from the following:

* Midlands Regional Trade Statistics data 2014-2016
* 2015 UK VAT Registered Importer and Exporter Population
* Overseas Trade Statistics broken down by English Growth Hub areas - EU and non-EU Exports 2015
* UK Trade in Goods by Business Characteristics - Experimental Statistics
* 2010 Quinquennial Review of the UK Ancillary Cost Survey (ACS) full report
* Quality Report
* UK Statistics article
* 2009 Intrastat Triennial Final Report
* EDICOM Technical Implementation Report
* 2005 Intrastat Triennial Review V2.0
* EDICOM Report
* EDICOM Report

In [2]:
scraper.select_dataset(title=lambda x: x.startswith('UK Trade in Goods by Business Characteristics'))
scraper

## UK Trade in Goods by Business Characteristics - Experimental Statistics

### Distributions

1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_2015.pdf))
1. IDBR OTS tables 2015 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_tables_2015.xls))
1. IDBR OTS Country data (expert users) 2015 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_%20Country_data_expert_2015.xls))
1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_2014.pdf))
1. IDBR OTS tables 2014 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_tables_2014.xls))
1. IDBR OTS Country data (expert users) 2014 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_%20Country_data_expert_2014.xls))
1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_2013.pdf))
1. IDBR OTS tables 2013 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_tables_2013.xls))
1. IDBR OTS Country data (expert users) 2013 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/Documents/Reports/IDBR_OTS_Country_data%20_expert_2013.xls))
1. UK Trade in Goods by Business Characteristics - Experimental Statistics ([application/pdf](https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Documents/IDBR_OTS_2012.pdf))
1. IDBR OTS tables 2009 - 2012 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Documents/IDBR_OTS_tables_2009_to_2012.xls))
1. IDBR OTS Country data (expert users) 2009 - 2012 ([MS Excel Spreadsheet](https://www.uktradeinfo.com/Statistics/OverseasTradeStatistics/AboutOverseastradeStatistics/Documents/IDBR_OTS_Country_2009_to_2012.xls))


In [3]:
idbrs = sorted(
    [dist for dist in scraper.distributions if dist.title.startswith('IDBR OTS tables')],
    key=lambda d: d.title, reverse=True)
idbr = idbrs[0]
display(idbr.title)
tabs = {tab.name: tab for tab in idbr.as_databaker()}
tabs.keys()

'IDBR OTS tables 2015'

dict_keys(['Notes and Contents', 'Industry Group', 'Age Group', 'Employee Size', 'Industry_Age', 'Industry_EmployeeSize', 'EmployeeSize_Age', 'Metadata'])

In [4]:
final_table = pd.DataFrame()

In [15]:
%%capture
%run "Business count by Age of Business.ipynb"
final_table = pd.concat([final_table, new_table])

%run "Business count by Employee Size.ipynb"
final_table = pd.concat([final_table, new_table])

%run "Business count by Industry Group.ipynb"
final_table = pd.concat([final_table, new_table])

%run "Employee count for Businesses by Age of Business.ipynb"
final_table = pd.concat([final_table, new_table])

%run "Employee count for Businesses by Employee Size.ipynb"
final_table = pd.concat([final_table, new_table])

%run "Employee count for Businesses by Industry Group.ipynb"
final_table = pd.concat([final_table, new_table])

%run "Total value of UK trade by Age of Business.ipynb"
final_table = pd.concat([final_table, new_table])

%run "Total value of UK trade by Employee Size.ipynb"
final_table = pd.concat([final_table, new_table])

%run "Total value of UK trade by Industry Group.ipynb"
final_table = pd.concat([final_table, new_table])

%run "TRADE IN GOODS STATISTICS -Business Count.ipynb"
final_table = pd.concat([final_table, new_table])

%run "TRADE IN GOODS STATISTICS -Employee Count and age business.ipynb"
final_table = pd.concat([final_table, new_table])

%run "TRADE IN GOODS STATISTICS -Employee Count and age employee count.ipynb"
final_table = pd.concat([final_table, new_table])

%run "TRADE IN GOODS STATISTICS -Employee Count and age.ipynb"
final_table = pd.concat([final_table, new_table])

%run "TRADE IN GOODS STATISTICS -Employee Count.ipynb"
final_table = pd.concat([final_table, new_table])

%run "TRADE IN GOODS STATISTICS -total value of UK Trade.ipynb"
final_table = pd.concat([final_table, new_table])

%run "TRADE IN GOODS STATISTICS_Employee size_Businesses.ipynb"
final_table = pd.concat([final_table, new_table])

%run "TRADE IN GOODS STATISTICS_Employee size_Employee count.ipynb"
final_table = pd.concat([final_table, new_table])

%run "TRADE IN GOODS STATISTICS_Employee size_value.ipynb"
final_table = pd.concat([final_table, new_table])


TRADE IN GOODS STATISTICS MATCHED WITH REGISTERED BUSINESSES FROM THE INTER-DEPARTMENTAL BUSINESS REGISTER,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,
Release Date 24/11/2016,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,
Total value of UK trade by Age of Business,,,,£ millions,,,,,,,,,,,,
,,Exports,,Imports,,,,,,,,,,,,
Age (years),,2015.0,,2015.0,,,,,,,,,,,,
,,,,,,,,,,,,,,,,
0 to 1,,5820.0,,15381.0,,,,,,,,,,,,
2 to 3,,8125.0,,20435.0,,,,,,,,,,,,


/home/alex/.virtualenvs/databaker/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


TRADE IN GOODS STATISTICS MATCHED WITH REGISTERED BUSINESSES FROM THE INTER-DEPARTMENTAL BUSINESS REGISTER,,,,,,,,
,,,,,,,,
Release Date 24/11/2016,,,,,,,,
,,,,,,,,
Total value of UK trade by Employee Size,,,,£ millions,,,,
,,Exports,,Imports,,,,
Employee Size,,2015.0,,2015.0,,,,
,,,,,,,,
0.0,,26919.0,,44923.0,,,,
1 to 9,,16369.0,,26778.0,,,,
10 to 49,,24266.0,,48100.0,,,,


/home/alex/.virtualenvs/databaker/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


TRADE IN GOODS STATISTICS MATCHED WITH REGISTERED BUSINESSES FROM THE INTER-DEPARTMENTAL BUSINESS REGISTER,,,,,,,,
,,,,,,,,
Release Date 24/11/2016,,,,,,,,
,,,,,,,,
Total value of UK trade by Industry Group,,,,£ millions,,,,
,,Exports,,Imports,,,,
Industry Group,,2015.0,,2015.0,,,,
,,,,,,,,
Group 1 Agriculture and Food,,15178.0,,18388.0,,,,
"Group 2 Mining, Petroleum products and Waste",,17836.0,,23226.0,,,,
Group 3 Chemicals,,13564.0,,9467.0,,,,


...

/home/alex/.virtualenvs/databaker/lib/python3.7/site-packages/ipykernel_launcher.py:47: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



TRADE IN GOODS STATISTICS MATCHED WITH REGISTERED BUSINESSES FROM THE INTER-DEPARTMENTAL BUSINESS REGISTER,,,,,,,,,,,,,
,,,,,,,,,,,,,
Release Date 24/11/2016,,,,,,,,,,,,,
,,,,,,,,,,,,,
"Total value of UK Trade, Business count and Employee count, by Industry group and Employee Size group for Imports and Exports",,,,,,,,,,,,,
,,,Exports,,,,Imports,,,,,,
,,,2015.0,,,,2015.0,,,,,,
Industry Group,Employees,,Value £ m,Business Count,Employee Count,,Value £ m,Business Count,Employee Count,,,,
,,,,,,,,,,,,,
Group 1 Agriculture and Food,0.0,,121.0,647.0,0.0,,588.0,1841.0,0.0,,,,
,1 to 9,,439.0,2725.0,10464.0,,594.0,4846.0,17472.0,,,,


/home/alex/.virtualenvs/databaker/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



TRADE IN GOODS STATISTICS MATCHED WITH REGISTERED BUSINESSES FROM THE INTER-DEPARTMENTAL BUSINESS REGISTER,,,,,,,,,,,,,
,,,,,,,,,,,,,
Release Date 24/11/2016,,,,,,,,,,,,,
,,,,,,,,,,,,,
"Total value of UK Trade, Business count and Employee count, by Industry group and Employee Size group for Imports and Exports",,,,,,,,,,,,,
,,,Exports,,,,Imports,,,,,,
,,,2015.0,,,,2015.0,,,,,,
Industry Group,Employees,,Value £ m,Business Count,Employee Count,,Value £ m,Business Count,Employee Count,,,,
,,,,,,,,,,,,,
Group 1 Agriculture and Food,0.0,,121.0,647.0,0.0,,588.0,1841.0,0.0,,,,
,1 to 9,,439.0,2725.0,10464.0,,594.0,4846.0,17472.0,,,,


/home/alex/.virtualenvs/databaker/lib/python3.7/site-packages/ipykernel_launcher.py:53: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [16]:
final_table.fillna('Total', inplace = True)

In [17]:
final_table.head()

,Age of Business,Employment,Flow,Geography,HMRC Industry,Measure Type,Unit,Value,Year
0,0 to 1 years,Total,Exports,K02000001,Total,Count,Businesses,10515,2015
1,0 to 1 years,Total,Imports,K02000001,Total,Count,Businesses,21541,2015
2,2 to 3 years,Total,Exports,K02000001,Total,Count,Businesses,13308,2015
3,2 to 3 years,Total,Imports,K02000001,Total,Count,Businesses,22043,2015
4,4 to 5 years,Total,Exports,K02000001,Total,Count,Businesses,12088,2015


In [18]:
final_table['Employment'].unique()

array(['Total', '0 employees', '1 to 9 employees', '10 to 49 employees',
       '50 to 249 employees', '250 + employees', 'Unknown employees',
       'Total employees', 'Grand Total employees', 'employees'],
      dtype=object)

In [19]:
def user_perc(x):
    
    if str(x) == 'employees':
        return 'Total employees'
    else:
        return x
    
final_table['Employment'] = final_table.apply(lambda row: user_perc(row['Employment']), axis = 1)


In [20]:
final_table['Employment'] = final_table['Employment'].map(lambda cell: cell.replace('Grand Total employees', 'Total employees'))

In [21]:
final_table['Flow'].unique()

array(['Exports', 'Imports', 'Export', 'Import'], dtype=object)

In [22]:
final_table['Flow'] = final_table['Flow'].map(lambda cell: cell.replace('Export', 'Exports'))
final_table['Flow'] = final_table['Flow'].map(lambda cell: cell.replace('Import', 'Imports'))

In [23]:
final_table['HMRC Industry'].unique()

array(['Total', 'group-1', 'group-2', 'group-3', 'group-4', 'group-5',
       'group-6', 'group-7', 'group-8', 'group-9', 'group-10', 'Unknown'],
      dtype=object)

In [24]:
final_table['HMRC Industry'] = final_table['HMRC Industry'].map(lambda cell: cell.replace('Agriculture  Food', 'Agriculture and Food'))

In [25]:
final_table['HMRC Industry'] = final_table['HMRC Industry'].map(lambda cell: cell.replace('Mining, Petroleum products  Waste', 'Mining, Petroleum products and Waste'))

In [26]:
final_table['HMRC Industry'] = final_table['HMRC Industry'].map(lambda cell: cell.replace('Electronic  Electrical equipment', 'Electronic and Electrical equipment'))

In [27]:
final_table['HMRC Industry'] = final_table['HMRC Industry'].map(lambda cell: cell.replace('Machinery  equipment n.e.s', 'Machinery and equipment n.e.s'))

In [28]:
final_table['HMRC Industry'] = final_table['HMRC Industry'].map(lambda cell: cell.replace('Aerospace  related machinery', 'Aerospace and related machinery'))

In [29]:
final_table['HMRC Industry'] = final_table['HMRC Industry'].map(lambda cell: cell.replace('Total', 'total'))

In [30]:
final_table['HMRC Industry'] = final_table['HMRC Industry'].map(lambda cell: cell.replace('Unknown', 'unknown'))

In [31]:
final_table['Unit'].unique()

array(['Businesses', 'Employees', '£ Million'], dtype=object)

In [32]:
final_table['Measure Type'].unique()

array(['Count', 'Total Turnover'], dtype=object)

In [33]:
final_table['Year'].unique()

array(['2015'], dtype=object)

In [34]:
final_table['Age of Business'].unique()

array(['0 to 1 years', '2 to 3 years', '4 to 5 years', '6 to 9 years',
       '10 to 20 years', '20 + years', 'Unknown years', 'Total years',
       'Total', ' years', '6 to 9  years', 'years'], dtype=object)

In [35]:
final_table['Age of Business'] = final_table['Age of Business'].map(lambda cell: cell.replace('Total years', 'total'))

In [36]:
final_table['Age of Business'] = final_table['Age of Business'].map(lambda cell: cell.replace('Unknown years', 'unknown'))

In [37]:
# final_table['Age of Business'] = final_table['Age of Business'].map(lambda cell: cell.replace('years', 'total'))

In [38]:
def user_perc2(x):
    
    if str(x) == '6 to 9  years':
        return '6 to 9 years'
    else:
        return x.strip()
    
final_table['Age of Business'] = final_table.apply(lambda row: user_perc2(row['Age of Business']), axis = 1)

In [39]:
def user_perc3(x):
    
    if str(x) == 'years':
        return 'Total'
    else:
        return x.strip()
    
final_table['Age of Business'] = final_table.apply(lambda row: user_perc3(row['Age of Business']), axis = 1)

In [40]:
final_table['Age of Business'].unique()

array(['0 to 1 years', '2 to 3 years', '4 to 5 years', '6 to 9 years',
       '10 to 20 years', '20 + years', 'unknown', 'total', 'Total'],
      dtype=object)

In [41]:
final_table = final_table[['Geography','Year','Employment','Flow','Age of Business','HMRC Industry','Measure Type','Value','Unit']]

In [42]:
final_table.head()

,Geography,Year,Employment,Flow,Age of Business,HMRC Industry,Measure Type,Value,Unit
0,K02000001,2015,Total,Exportss,0 to 1 years,total,Count,10515,Businesses
1,K02000001,2015,Total,Importss,0 to 1 years,total,Count,21541,Businesses
2,K02000001,2015,Total,Exportss,2 to 3 years,total,Count,13308,Businesses
3,K02000001,2015,Total,Importss,2 to 3 years,total,Count,22043,Businesses
4,K02000001,2015,Total,Exportss,4 to 5 years,total,Count,12088,Businesses


In [43]:
final_table.tail()

,Geography,Year,Employment,Flow,Age of Business,HMRC Industry,Measure Type,Value,Unit
119,K02000001,2015,Unknown employees,Imports,Total,group-10,Total Turnover,4288,£ Million
120,K02000001,2015,Unknown employees,Exports,Total,unknown,Total Turnover,11160,£ Million
121,K02000001,2015,Unknown employees,Imports,Total,unknown,Total Turnover,6459,£ Million
122,K02000001,2015,Total employees,Exports,Total,total,Total Turnover,304768,£ Million
123,K02000001,2015,Total employees,Imports,Total,total,Total Turnover,411033,£ Million


In [44]:
out = Path('out')
out.mkdir(exist_ok=True)
final_table.to_csv(out / 'observations.csv', index = False)

In [45]:
scraper.dataset.family = 'migration'
scraper.theme = 'migration'
with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())